In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
# input data: 주소, 건물 정보. 토지 정보는 자동으로 가져와야 함. 토지 정보를 자동으로 가져오려면 오리지널 토지특성정보 &
# 토지이용계획정보, 좌표는 api에서 다운로드해야함.

In [3]:
# 1. 오리지널 토지특성정보, 토지이용계획정보 & 공시지가
# 2. 좌표
# 3. 건물 정보

In [4]:
# 1. 오리지널 토지특성정보 & 토지이용계획정보

In [5]:
p_land_specs_df = pd.read_csv('./토지특성정보/land_specs_production_ver_1.csv')

In [6]:
print(p_land_specs_df.shape)
p_land_specs_df.head()

(983625, 9)


,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지번주소
0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,서울특별시 강남구 개포동 100-0
1,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 101-0
2,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 102-0
3,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,서울특별시 강남구 개포동 103-0
4,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,서울특별시 강남구 개포동 104-1


In [7]:
p_land_plans_df = pd.read_csv('./토지이용계획정보/seoul_land_plans_production_ver_1.csv')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,2

In [8]:
print(p_land_plans_df.shape)
p_land_plans_df.head()

(983564, 350)


,지번주소,년,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,4대문안,가로구역별 최고높이 제한지역,가스공급설비,가스공급시설,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고가도로,고도지구,고등학교,고속철도,고압선,공간시설미분류,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공시설용지,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공설화장시설,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원문화유산지구,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항,공항소음피해예상지역,공항소음피해지역,공항시설보호지구,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광역복합환승센터,광장,교육환경보호구역,교차점광장,교통광장,교통운수시설미분류,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,군사기지 및 군사시설 보호구역,군사시설 보호구역,궤도,근린공원,근린광장,근린상업지역,기업형임대주택 공급촉진지구,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타도시방재시설,기타문화시설,기타방송통신시설,기타보건위생시설,기타사회복지시설,기타수도시설,기타시장시설,기타열공급설비,기타용도지역지구기타,기타용도지역지구미분류,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타전기공급설비,기타주차장시설,기타철도시설,기타폐기물처리시설,기타하수도시설,기타학교시설,기타환경기초시설,노외주차장,녹지,농수산물공판장및농수산물종합유통센터,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,등록문화재구역,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,미관지구기타,박물관,방송통신시설,방수설비,방재지구,방화지구,배수시설,변전소(전원개발사업구역),변전시설,보건위생시설미분류,보전녹지지역,보전산지,보전임지,보조간선도로,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제1구역(전술),비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업기술단지,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,생태·경관보전지역,생태·경관완충보전구역,생태·경관핵심보전구역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,소하천예정지,수도공급시설,수질오염방지시설,수평표면구역,시·도 생태·경관보전지역,시·도야생생물보호구역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,시험림구역,아파트지구,액화석유가스충전시설,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,온천원보호지구,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,운동장,원추표면구역,원형보존지,유류저장및송유설비,유수시설,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반광장,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차운전학원,자동차전용도로,자동차정류장,자연경관지구,자연공원용도지구기타,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,재해위험지구기타,재활용시설,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(방공기지 : 1km),제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합운동장,종합의료시설,주간선도로,주거용지,주거환경개선지구,주요시설광장,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보존지구,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,집산도로,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,취수시설,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,토지형질변경규제지역,통제보호구역,통제보호구역(민통선이남:300m),통제보호구역(방공기지:500m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,하천미분류,학교,학교시설보호지구,학교이적지,학교환경위생 정화구역,현상변경허가 대상구역,홍수관리구역,화장장,환경정비구역
0,서울특별시 강남구 개포동 100-0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,접함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,접함,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,접함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,저촉,NaN,NaN,접함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,포함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [9]:
land_prices_df = pd.read_csv('./공시지가/land_prices_production_ver.csv')

In [10]:
print(land_prices_df.shape)
land_prices_df.head()

(903493, 5)


,지번주소,특수지구분명,기준년도,공시지가,표준지여부
0,서울특별시 강남구 개포동 100-0,일반,2020,396000,N
1,서울특별시 강남구 개포동 101-0,일반,2020,475200,N
2,서울특별시 강남구 개포동 102-0,일반,2020,475200,N
3,서울특별시 강남구 개포동 103-0,일반,2020,309700,N
4,서울특별시 강남구 개포동 104-1,일반,2020,394600,N


In [11]:
# 2. 좌표

In [12]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDEyNDExMDM0MDExMDczNjE='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 4월 24일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], 'x좌표':[], 'y좌표':[]})
    
    for i, address in tqdm(enumerate(addr_list)):        
        address_no_zero = address.replace('-0','')
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1'\
                                       + '&keyword=' + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        roadname_info = roadname_result.json()['results']['juso']
        
        # 지번주소에 매칭하는 도로명주소가 없을 경우, 근처 지번주소의 도로명주소로 대체
        
        if roadname_info == []:
            splitted = address_no_zero.split('-')
            space_splitted = address_no_zero.split(' ')
            
            if len(splitted) == 1:
                bonbun = int(space_splitted[3])
                bonbun_positive = bonbun
                bonbun_negative = bonbun
                
                while roadname_info == []:
                
                    bonbun_positive += 1
                    bonbun_negative -= 1
                    
                    addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                    + str(bonbun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                   + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                   + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                        + str(bonbun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                       + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                       + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                
                
            else:
                bubun = int(splitted[1])
                bubun_positive = bubun
                bubun_negative = bubun
                
                count = 0
                while roadname_info == []:
                    
                    bubun_positive += 1
                    bubun_negative -= 1
                    
                    addr = splitted[0] + '-' + str(bubun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                   + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                   + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = splitted[0] + '-' + str(bubun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                       + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                       + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                    
                    count += 1
                    
                    if count > 10:
                        
                        addr = splitted[0]
                        
                        space_splitted = addr.split(' ')
                        
                        bonbun = int(space_splitted[3])
                        bonbun_positive = bonbun
                        bonbun_negative = bonbun
                        
                        while roadname_info == []:

                            bonbun_positive += 1
                            bonbun_negative -= 1

                            addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                            + str(bonbun_positive)
                            
                            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1'\
                                                           + '&countPerPage=1' + '&keyword=' + addr + '&resultType=json'\
                                                           + '&hstryYn=Y')
                            roadname_info = roadname_result.json()['results']['juso']

                            if roadname_info == []:

                                addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' '\
                                + str(bonbun_negative)
                                
                                roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey\
                                                               + '&currentPage=1' + '&countPerPage=1' + '&keyword='\
                                                               + addr + '&resultType=json' + '&hstryYn=Y')
                                roadname_info = roadname_result.json()['results']['juso']
                                
        roadname_info = roadname_info[0]                                
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [13]:
training_data_df = pd.read_csv('./Training/training_data_ver_16/training_data_ver_16.csv')

KeyboardInterrupt: 

In [ ]:
print(training_data_df.shape)
training_data_df.head()

In [ ]:
p_land_specs_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in p_land_specs_df.columns]

In [ ]:
p_land_specs_df.columns

In [ ]:
p_land_plans_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in p_land_plans_df.columns]

In [ ]:
p_land_plans_df.columns

In [17]:
bdinfo_cols = []
for col in training_data_df.columns:
    if col not in p_land_specs_df:
        if col not in p_land_plans_df:
            bdinfo_cols.append(col)

In [18]:
len(bdinfo_cols)

109

In [19]:
bdinfo_cols

['전용면적',
 '금액(만원)',
 '층',
 '건축년도',
 '가격면적',
 'target_log_transformed',
 'targetarea_log_transformed',
 '주택가격지수',
 '전용면적_mean',
 '전용면적_min',
 '전용면적_max',
 '건물나이',
 'year_linear',
 'month_sin',
 'month_cos',
 'xnorm',
 'ynorm',
 'x_2nd',
 'y_2nd',
 'x_sin',
 'y_sin',
 'x_cos',
 'y_cos',
 'xy',
 'trade_type',
 '공시지가(만원)',
 '표준지여부',
 '철근콘크리트구조_fraction',
 '철골철근콘크리트구조_fraction',
 '벽돌구조_fraction',
 '프리케스트콘크리트구조_fraction',
 '경량철골구조_fraction',
 '철골콘크리트구조_fraction',
 '일반철골구조_fraction',
 '블록구조_fraction',
 '일반목구조_fraction',
 '기타조적구조_fraction',
 '기타강구조_fraction',
 '강파이프구조_fraction',
 '기타구조_fraction',
 '석구조_fraction',
 '기타콘크리트구조_fraction',
 '라멘조_fraction',
 '조립식판넬조_fraction',
 '주_용도_공동주택_fraction',
 '주_용도_종교시설_fraction',
 '주_용도_제2종근린생활시설_fraction',
 '주_용도_단독주택_fraction',
 '주_용도_제1종근린생활시설_fraction',
 '주_용도_교육연구시설_fraction',
 '주_용도_판매시설_fraction',
 '주_용도_노유자시설_fraction',
 '주_용도_자동차관련시설_fraction',
 '주_용도_업무시설_fraction',
 '주_용도_운동시설_fraction',
 '주_용도_창고시설_fraction',
 '주_용도_문화및집회시설_fraction',
 '주_용도_숙박시

In [22]:
bdinfo_cols.remove('금액(만원)')
bdinfo_cols.remove('가격면적')
bdinfo_cols.remove('target_log_transformed')
bdinfo_cols.remove('targetarea_log_transformed')
bdinfo_cols.remove('주택가격지수')
bdinfo_cols.remove('xnorm')
bdinfo_cols.remove('ynorm')
bdinfo_cols.remove('x_2nd')
bdinfo_cols.remove('y_2nd')
bdinfo_cols.remove('x_sin')
bdinfo_cols.remove('y_sin')
bdinfo_cols.remove('x_cos')
bdinfo_cols.remove('y_cos')
bdinfo_cols.remove('xy')

In [23]:
len(bdinfo_cols)

95

In [24]:
bdinfo_cols

['전용면적',
 '층',
 '건축년도',
 '전용면적_mean',
 '전용면적_min',
 '전용면적_max',
 '건물나이',
 'year_linear',
 'month_sin',
 'month_cos',
 'trade_type',
 '공시지가(만원)',
 '표준지여부',
 '철근콘크리트구조_fraction',
 '철골철근콘크리트구조_fraction',
 '벽돌구조_fraction',
 '프리케스트콘크리트구조_fraction',
 '경량철골구조_fraction',
 '철골콘크리트구조_fraction',
 '일반철골구조_fraction',
 '블록구조_fraction',
 '일반목구조_fraction',
 '기타조적구조_fraction',
 '기타강구조_fraction',
 '강파이프구조_fraction',
 '기타구조_fraction',
 '석구조_fraction',
 '기타콘크리트구조_fraction',
 '라멘조_fraction',
 '조립식판넬조_fraction',
 '주_용도_공동주택_fraction',
 '주_용도_종교시설_fraction',
 '주_용도_제2종근린생활시설_fraction',
 '주_용도_단독주택_fraction',
 '주_용도_제1종근린생활시설_fraction',
 '주_용도_교육연구시설_fraction',
 '주_용도_판매시설_fraction',
 '주_용도_노유자시설_fraction',
 '주_용도_자동차관련시설_fraction',
 '주_용도_업무시설_fraction',
 '주_용도_운동시설_fraction',
 '주_용도_창고시설_fraction',
 '주_용도_문화및집회시설_fraction',
 '주_용도_숙박시설_fraction',
 '주_용도_근린생활시설_fraction',
 '주_용도_위험물저장및처리시설_fraction',
 '주_용도_동.식물관련시설_fraction',
 '주_용도_의료시설_fraction',
 '주_용도_공장_fraction',
 '주_용도_관광휴게시설_fraction',
 '주_용도_수련시설_f

In [25]:
bdinfo_cols.remove('year_linear')
bdinfo_cols.remove('month_sin')
bdinfo_cols.remove('month_cos')
bdinfo_cols.remove('공시지가(만원)')
bdinfo_cols.remove('표준지여부')

In [26]:
len(bdinfo_cols)

90

In [27]:
bdinfo_cols

['전용면적',
 '층',
 '건축년도',
 '전용면적_mean',
 '전용면적_min',
 '전용면적_max',
 '건물나이',
 'trade_type',
 '철근콘크리트구조_fraction',
 '철골철근콘크리트구조_fraction',
 '벽돌구조_fraction',
 '프리케스트콘크리트구조_fraction',
 '경량철골구조_fraction',
 '철골콘크리트구조_fraction',
 '일반철골구조_fraction',
 '블록구조_fraction',
 '일반목구조_fraction',
 '기타조적구조_fraction',
 '기타강구조_fraction',
 '강파이프구조_fraction',
 '기타구조_fraction',
 '석구조_fraction',
 '기타콘크리트구조_fraction',
 '라멘조_fraction',
 '조립식판넬조_fraction',
 '주_용도_공동주택_fraction',
 '주_용도_종교시설_fraction',
 '주_용도_제2종근린생활시설_fraction',
 '주_용도_단독주택_fraction',
 '주_용도_제1종근린생활시설_fraction',
 '주_용도_교육연구시설_fraction',
 '주_용도_판매시설_fraction',
 '주_용도_노유자시설_fraction',
 '주_용도_자동차관련시설_fraction',
 '주_용도_업무시설_fraction',
 '주_용도_운동시설_fraction',
 '주_용도_창고시설_fraction',
 '주_용도_문화및집회시설_fraction',
 '주_용도_숙박시설_fraction',
 '주_용도_근린생활시설_fraction',
 '주_용도_위험물저장및처리시설_fraction',
 '주_용도_동.식물관련시설_fraction',
 '주_용도_의료시설_fraction',
 '주_용도_공장_fraction',
 '주_용도_관광휴게시설_fraction',
 '주_용도_수련시설_fraction',
 '주_용도_판매및영업시설_fraction',
 '주_용도_교육연구및복지시설_fraction',
 '지

In [28]:
features_to_fill = bdinfo_cols

In [29]:
features_to_fill_df = pd.DataFrame({'목록':features_to_fill})

In [30]:
print(features_to_fill_df.shape)

(90, 1)


In [31]:
features_to_fill_df

,목록
0,전용면적
1,층
2,건축년도
3,전용면적_mean
4,전용면적_min
5,전용면적_max
6,건물나이
7,trade_type
8,철근콘크리트구조_fraction
9,철골철근콘크리트구조_fraction


In [12]:
features_to_fill_df.to_excel('./Production_Prep/직접작성필요목록.xlsx', index=False)

In [ ]:
features_to_fill_df = pd.read

In [ ]:
def prediction_pipeline(addr_list, land_specs_df, land_plans_df)

In [ ]:
basedir = './prepped_data/'

In [ ]:
%%time
main_data_df = pd.read_csv(basedir + 'main_data_monthly_converted.csv')
print(main_data_df.shape)
main_data_df.head()

In [ ]:
%%time
land_prices_df = pd.read_csv(basedir + 'land_prices_ver_2.csv')
land_prices_df.shape

In [ ]:
%%time
land_specs_df = pd.read_csv(basedir + 'land_specs_ver_3.csv')
land_specs_df.shape

In [ ]:
%%time
land_plans_df = pd.read_csv(basedir + 'land_plans_ver_3.csv')
land_plans_df.shape

In [ ]:
land_plans_df.fillna('해당사항없음', inplace=True)

In [ ]:
%%time
bdinfo_df = pd.read_csv(basedir + 'bdinfo_features_structure_purpose_roof.csv')
bdinfo_df.shape

In [ ]:
%%time
bdinfo2_df = pd.read_csv(basedir + 'new_building_features_2021_0209.csv')
print(bdinfo2_df.shape)
bdinfo2_df.head()

In [ ]:
main_data_df.rename(columns={'전용면적(㎡)':'전용면적'}, inplace=True)

In [ ]:
%%time
main_data_df['전용면적_mean'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('mean')['전용면적']
main_data_df['전용면적_min'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('min')['전용면적']
main_data_df['전용면적_max'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('max')['전용면적']

In [ ]:
main_data_df['가격/면적'] = main_data_df['금액(만원)'] / main_data_df['전용면적']

In [ ]:
main_data_df['건물나이'] = main_data_df['년'] - main_data_df['건축년도']
main_data_df['건물나이'].describe()

In [ ]:
main_data_df['year_linear'] = (main_data_df['년'] - main_data_df['년'].min()) + ((main_data_df['월']-1)/12)

In [ ]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [ ]:
main_data_df['month_sin'] = main_data_df['월'].apply(month_circular_sine_func)
main_data_df['month_cos'] = main_data_df['월'].apply(month_circular_cosine_func)

In [ ]:
main_data_df.drop(columns=['월', '계약일'], inplace=True)

In [ ]:
xmax = main_data_df['X좌표'].max()
xmin = main_data_df['X좌표'].min()
ymax = main_data_df['Y좌표'].max()
ymin = main_data_df['Y좌표'].min()

In [ ]:
coor_minmax_df = pd.DataFrame({'xmax':xmax, 'xmin':xmin, 'ymax':ymax, 'ymin':ymin}, index=[0])
coor_minmax_df.to_csv('./Training/training_data_ver_14/coor_minmax.csv', index=False)
coor_minmax_df

In [ ]:
main_data_df['xnorm'] = (main_data_df['X좌표'] - xmin - ((xmax - xmin)/2))
main_data_df['ynorm'] = (main_data_df['Y좌표'] - ymin - ((ymax - ymin)/2))
main_data_df['x_2nd'] = main_data_df['xnorm']**2 / 2
main_data_df['y_2nd'] = main_data_df['ynorm']**2 / 2
main_data_df['x_sin'] = main_data_df['xnorm'].apply(np.sin)
main_data_df['y_sin'] = main_data_df['ynorm'].apply(np.sin)
main_data_df['x_cos'] = main_data_df['xnorm'].apply(np.cos)
main_data_df['y_cos'] = main_data_df['ynorm'].apply(np.cos)
main_data_df['xy'] = main_data_df['xnorm'] * main_data_df['ynorm']

In [ ]:
main_data_df.drop(columns=['X좌표', 'Y좌표'], inplace=True)

In [ ]:
main_data_df['trade_type'] = main_data_df['건물종류'] + '_' + main_data_df['전월세매매구분']
main_data_df.drop(columns=['건물종류', '전월세매매구분'], inplace=True)

In [ ]:
merge1_df = main_data_df.merge(land_prices_df, on=['지번주소', '년'])
merge1_df.shape

In [ ]:
merge1_df['공시지가'] = merge1_df['공시지가'] / 10000
merge1_df.rename(columns={'공시지가':'공시지가(만원)'}, inplace=True)

In [ ]:
bdinfo_df.drop(columns=['사용승인년도'], inplace=True)
bdinfo_df.rename(columns={'기준년도':'년'}, inplace=True)

In [ ]:
merge1_df.shape

In [ ]:
merge2_df = merge1_df.merge(bdinfo_df, on=['지번주소', '년'])
merge2_df.shape

In [ ]:
%%time
bdinfo3_df = pd.read_csv(basedir + 'extracted_building_features.csv')
bdinfo3_df = bdinfo3_df[['지번주소', '기준년도'] + [col for col in bdinfo3_df.columns if '층' in col]]
print(bdinfo3_df.shape)
bdinfo3_df.head()

In [ ]:
bdinfo3_df.rename(columns={'기준년도':'년'}, inplace=True)

In [ ]:
bdinfo2_df.head()

In [ ]:
bdinfo2_df.drop(columns=['사용승인년도', '대장종류'], inplace=True)

In [ ]:
bdinfo2_df.rename(columns={'기준년도':'년'}, inplace=True)

In [ ]:
to_drop_df = bdinfo2_df[(bdinfo2_df['부속건축물수'] > 0) & (bdinfo2_df['부속건축물면적'] == 0)]
print(to_drop_df.shape)

In [ ]:
bdinfo2_df.drop(to_drop_df.index, inplace=True)

In [ ]:
merge2_df = merge2_df.merge(bdinfo2_df, on=['지번주소', '년'])
print(merge2_df.shape)

In [ ]:
merge2_df = merge2_df.merge(bdinfo3_df, on=['지번주소', '년'])

In [ ]:
merge2_df.head()

In [ ]:
merge2_df['최고층-층'] = merge2_df['층max'] - merge2_df['층']
merge2_df['최고층-층'].describe()

In [ ]:
negative_df = merge2_df[merge2_df['최고층-층'] < 0]
negative_df.shape

In [ ]:
negative_df['지번주소'].nunique()

In [ ]:
idx_to_drop = []
for addr in tqdm(negative_df['지번주소'].unique()):
    addr_df = merge2_df[merge2_df['지번주소'] == addr]
    addr2_df = negative_df[negative_df['지번주소'] == addr]
    min_year = addr2_df['년'].min()
    df_to_drop = addr_df[addr_df['년'] >= min_year]
    for idx in df_to_drop.index:
        idx_to_drop.append(idx)

In [ ]:
len(list(set(idx_to_drop)))

In [ ]:
merge2_df.drop(idx_to_drop, inplace=True)

In [ ]:
merge2_df.shape

In [ ]:
merge2_df['층norm'] = merge2_df['층'] / merge2_df['층max']

In [ ]:
merge2_df['층norm'].describe()

In [ ]:
merge2_df.drop(columns=['최고층-층'], inplace=True)

In [ ]:
merge2_df['지하세대있음'] = merge2_df['층'].apply(lambda x: 1 if x < 0 else 0)
merge2_df['지하세대있음'].value_counts()

In [ ]:
cols = [col for col in land_specs_df.columns if col != '공시지가']
cols

In [ ]:
merge2_df = merge2_df.merge(land_specs_df[cols], on=['지번주소', '년'])
merge2_df.shape

In [ ]:
merge2_df.drop(columns=['대지면적'], inplace=True)

In [ ]:
merge3_df = merge2_df.merge(land_plans_df, on=['지번주소', '년'])
merge3_df.shape

In [ ]:
merge3_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in merge3_df.columns]

In [ ]:
land_plans_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','').replace('ㆍ','_').replace(' ','_') for col in land_plans_df.columns]

In [ ]:
lp_catcols = land_plans_df.columns.tolist()[2:]
lp_catcols[:5]

In [ ]:
ls_catcols = [col for col in cols if col not in ['지번주소', '년', '토지면적']]
ls_catcols

In [ ]:
def building_age_feature_engi(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        
        if x < 0:
            return '건축전계약'
        elif x < 5:
            return '5년미만'
        elif x < 10:
            return '10년미만'
        elif x < 15:
            return '15년미만'
        elif x < 20:
            return '20년미만'
        elif x < 30:
            return '30년미만'
        elif x < 40:
            return '40년미만'
        else:
            return '40년이상'

In [ ]:
merge3_df = merge3_df[merge3_df['건물나이'] >= 0]
merge3_df.shape

In [ ]:
merge3_df['건물나이'] = merge3_df['건물나이'].apply(building_age_feature_engi)
merge3_df['건물나이'].value_counts()

In [ ]:
merge3_df.head()

In [ ]:
merge3_df.drop(columns=['년', '도로명주소'], inplace=True)

In [ ]:
cat_cols = ['trade_type', '표준지여부', '건물나이'] + ls_catcols + lp_catcols

In [ ]:
for col in tqdm(cat_cols):
    col_map = {}
    col_map_df = pd.DataFrame({'cat':merge3_df[col].unique(),'map':[i for i in range(len(merge3_df[col].unique()))]})
    for i in range(col_map_df.shape[0]):
        col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
    merge3_df[col] = merge3_df[col].map(col_map)
    col_map_df.to_csv('./Training/training_data_ver_16/feature_maps/{}_map.csv'.format(col))

In [ ]:
cat_cols_df = pd.DataFrame({'colname':cat_cols})
cat_cols_df.head()

In [ ]:
cat_cols_df.to_csv('./Training/training_data_ver_16/cat_cols.csv', index=False)

In [ ]:
cols_to_drop = []
for col in tqdm(merge3_df.columns):
    try:
        if merge3_df[col].std() == 0:
            cols_to_drop.append(col)
        else:
            pass
    except:
        pass

In [ ]:
len(cols_to_drop)

In [ ]:
cols_to_drop

In [ ]:
merge3_df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
%%time
merge3_df.to_csv('./Training/training_data_ver_16/training_data_ver_16.csv', index=False)

In [ ]:
merge3_df.shape